In [ ]:
import sys
import cv2
import torch
import pandas as pd
from tqdm import tqdm
import os
import dlib
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import interact, Select

In [ ]:
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh
IMAGE_FILES = []
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

right_list = [7,33,112,130,133,144,155,153,154,157,158,159,160,160,161,162,163, 173]
left_list = [249,255,341,359,382,373,374,380,381,381,384,385,386,387,388,390,398,466]


In [ ]:
import math
from mediapipe.framework.formats import landmark_pb2
from typing import List, Mapping, Optional, Tuple, Union
landmark_pb2.NormalizedLandmarkList
import numpy as np

_PRESENCE_THRESHOLD = 0.5
_VISIBILITY_THRESHOLD = 0.5
_BGR_CHANNELS = 3

ldmks_list = []


def _normalized_to_pixel_coordinates(
    normalized_x: float, normalized_y: float, image_width: int,
    image_height: int) -> Union[None, Tuple[int, int]]:
  """Converts normalized value pair to pixel coordinates."""

  # Checks if the float value is between 0 and 1.
  def is_valid_normalized_value(value: float) -> bool:
    return (value > 0 or math.isclose(0, value)) and (value < 1 or
                                                      math.isclose(1, value))

  if not (is_valid_normalized_value(normalized_x) and
          is_valid_normalized_value(normalized_y)):
    # TODO: Draw coordinates even if it's outside of the image bounds.
    return None
  x_px = min(math.floor(normalized_x * image_width), image_width - 1)
  y_px = min(math.floor(normalized_y * image_height), image_height - 1)
  return x_px, y_px

In [ ]:
read_parent_folder_path = '/Columbia Gaze Data Set'
save_csv_folder = ''
save_img_folder_path = ''
save_all_img_folder_path = os.path.join(save_img_folder_path, 'columbia_all')
error_img_folder_path = os.path.join(save_img_folder_path, 'error_img')
os.makedirs(save_csv_folder, exist_ok=True)
os.makedirs(save_img_folder_path, exist_ok=True)
os.makedirs(save_all_img_folder_path, exist_ok=True)
os.makedirs(error_img_folder_path, exist_ok=True)

In [ ]:
def convert_gaze_direction(name, alpha, beta):
    gt_gaze_x = np.sin(beta) * np.cos(alpha)
    gt_gaze_y = np.sin(alpha)
    gt_gaze_z = np.cos(alpha) * np.cos(beta)

    return [name, gt_gaze_x, gt_gaze_y, gt_gaze_z*-1]

In [ ]:
def show_img(img):
    print(img.shape)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()
    
def show_gaze_img(img, gaze):
    y , x, _ = img.shape
    x_2 = int(x/2)
    y_2 = int(y/2)

    img = cv2.circle(img, (x_2, y_2), 1, (0,0,255), thickness = 1)
    img = cv2.arrowedLine(img, (x_2, y_2), (x_2 + int(100*gaze[0]), y_2 + int(100*gaze[1])), (255, 0, 0), thickness=3)
    show_img(img)

In [ ]:
def img_cropper(img, ldmk_x, ldmk_y):
    if ldmk_y - 18 < 0:
        movement_h = int(round(18 - ldmk_y, 0))
    elif ldmk_y + 18 > img.shape[0]:
        movement_h = int(img.shape[0] - (ldmk_y + 18))
    else:
        movement_h = 0

    if ldmk_x - 30 < 0:
        movement_w = int(round(30 - ldmk_x, 0))
    elif ldmk_x + 30 > img.shape[1]:
        movement_w = int(round(img.shape[1] - (ldmk_x + 30)))
    else:
        movement_w = 0
    
    return img[int(round(ldmk_y-18, 0)+movement_h):int(round(ldmk_y+18, 0)+movement_h), int(round(ldmk_x-30, 0))+movement_w:int(round(ldmk_x+30,0))+movement_w]
    

In [ ]:
read_folder_list = sorted(os.listdir(read_parent_folder_path))
remove_file_name = '.DS_Store'
idx = read_folder_list.index(remove_file_name)
read_folder_list.pop(idx)
# print(read_folder_list)

for i in range(len(read_folder_list)):
    read_folder_name = read_folder_list[i]
    read_folder_path = os.path.join(read_parent_folder_path, read_folder_name)
    read_img_list = os.listdir(read_folder_path)
    personal_gaze_list = []
    save_personal_folder = os.path.join(save_img_folder_path, read_folder_name)
    save_folder_right = os.path.join(save_personal_folder, 'right')
    save_folder_left = os.path.join(save_personal_folder, 'left')
    save_personal_csv_name = read_folder_name + '.csv'
    save_personal_csv_path = os.path.join(save_csv_folder, save_personal_csv_name)
    print('save personal csv name: ', save_personal_csv_name)

In [ ]:
read_folder_list = sorted(os.listdir(read_parent_folder_path))
remove_file_name = '.DS_Store'
idx = read_folder_list.index(remove_file_name)
read_folder_list.pop(idx)
# print(read_folder_list)

img_path_list = []
all_gaze_list = []
pitch_yaw_list = []
all_name_cnt = 1
gaze_cnt = 1
error_img_cnt = 1
data_cnt = 1

for i in range(len(read_folder_list)):
    read_folder_name = read_folder_list[i]
    read_folder_path = os.path.join(read_parent_folder_path, read_folder_name)
    read_img_list = os.listdir(read_folder_path)
    personal_gaze_list = []
    save_personal_folder = os.path.join(save_img_folder_path, read_folder_name)
    save_folder_right = os.path.join(save_personal_folder, 'right')
    save_folder_left = os.path.join(save_personal_folder, 'left')
    os.makedirs(save_personal_folder, exist_ok=True)
    os.makedirs(save_folder_right, exist_ok=True)
    os.makedirs(save_folder_left, exist_ok=True)
    personal_name_cnt = 1

    for j in tqdm(range(len(read_img_list))):
        read_img_name = read_img_list[j]
        img_data_list = read_img_name.split('_')
        
        if len(img_data_list) == 5:
            '''Read Image Path'''
            img_name = read_img_name
            img_path = os.path.join(read_folder_path, read_img_name)
            # print(img_path)

            '''Gaze'''
            head_rot = float(img_data_list[2][:-1]) * -1
            vertical = float(img_data_list[3][:-1]) 
            # horizontal = float(img_data_list[4][:-5]) + head_rot
            horizontal = float(img_data_list[4][:-5])
            pitch = vertical * np.pi / 180
            yaw = horizontal * np.pi / 180
            gaze = convert_gaze_direction(img_name, -1*pitch, yaw)
            personal_gaze_list.append(gaze)
            all_gaze_list.append(gaze)
            # print(gaze_cnt)
            gaze_cnt += 1

            '''Detect Eye'''
            face_img = cv2.imread(img_path)
            face_img = cv2.resize(face_img, dsize=None, fx=0.15, fy=0.15)
            gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY) # convert to grayscale detector = dlib.get_frontal_face_detector()
            data_cnt += 1
            right_ldmks = []
            left_ldmks = []
            ldmks_list = []

            with mp_face_mesh.FaceMesh(
                static_image_mode=True,
                max_num_faces=1,
                refine_landmarks=True,
                min_detection_confidence=0.5) as face_mesh:

                image = face_img
                image_rows, image_cols, _ = image.shape
                results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                annotated_image = image.copy()

                if results.multi_face_landmarks == None:
                    error_img_name = str(error_img_cnt) + '.png'
                    error_img_path = os.path.join(error_img_folder_path, error_img_name)
                    cv2.imwrite(error_img_path, face_img)
                    error_img_cnt += 1
                else:
                    for face_landmarks in results.multi_face_landmarks:
                        mp_drawing.draw_landmarks(
                        image=annotated_image,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_TESSELATION,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles
                        .get_default_face_mesh_tesselation_style())

                    for idx, landmark in enumerate(face_landmarks.landmark):
                        if ((landmark.HasField('visibility') and
                            landmark.visibility < _VISIBILITY_THRESHOLD) or 
                            (landmark.HasField('presence') and
                            landmark.presence < _PRESENCE_THRESHOLD)):
                            continue
                        landmark_px = _normalized_to_pixel_coordinates(landmark.x, landmark.y,
                                                                    image_cols, image_rows)
                        ldmks_list.append(landmark_px)                                                
                        # print(landmark_px)

                    ldmks_array = np.array(ldmks_list)
                    # print(ldmks_array.shape)
                    img = face_img.copy()

                    if len(ldmks_array.shape) == 1:
                        for r_num in right_list:
                            if ldmks_array[r_num] != None:
                                right_ldmks.append(ldmks_array[r_num])
                        for l_num in left_list:
                            if ldmks_array[l_num] != None:
                                left_ldmks.append(ldmks_array[l_num])
                    
                    else:
                        for r_num in right_list:
                            if ldmks_array[r_num][0] != None:
                                right_ldmks.append(ldmks_array[r_num])
                        for l_num in left_list:
                            if ldmks_array[l_num][0] != None:
                                left_ldmks.append(ldmks_array[l_num])

                    right_ldmks_center = np.mean(right_ldmks, axis=0)
                    left_ldmks_center = np.mean(left_ldmks, axis=0)
                    output_img_w = 60
                    output_img_h = 36

                    save_img_right = img_cropper(gray, right_ldmks_center[0], right_ldmks_center[1])
                    save_img_left = img_cropper(gray, left_ldmks_center[0], left_ldmks_center[1])

                    save_img_name = str(personal_name_cnt) + '.png'

                    if save_img_right.shape[0] == 36 and save_img_right.shape[1] == 60:
                        save_img_right_path = os.path.join(save_folder_right, save_img_name)
                        cv2.imwrite(save_img_right_path, save_img_right)
                    else:
                        sys.exit('Error Right Image')

                    if save_img_left.shape[0] == 36 and save_img_left.shape[1] == 60:
                        save_img_left_path = os.path.join(save_folder_left, save_img_name)
                        save_all_img_name = str(all_name_cnt) + '.png'
                        save_img_all_path = os.path.join(save_all_img_folder_path, save_all_img_name)
                        cv2.imwrite(save_img_left_path, save_img_left)
                        cv2.imwrite(save_img_all_path, save_img_left)
                    else:
                        sys.exit('Error Left Image')                        

                    
                    # '''save all img folder'''
                    # save_all_img_name = str(all_name_cnt) + '.png'
                    # save_all_img_path = os.path.join(save_all_img_folder_path, save_all_img_name)
                    # cv2.imwrite(save_all_img_path, save_img)
                    all_name_cnt += 1                
                    personal_name_cnt += 1
                    
                    '''save_personal_img_folder'''
                    # save_personal_img_name = str(personal_name_cnt) + '.png'
                    # save_personal_img_right_path = os.path.join(save_img_right_path, save_personal_img_name)
                    # cv2.imwrite(save_personal_img_right_path, save_img_right)

                    # save_personal_img_left_path = os.path.join(save_img_left_path, save_personal_img_name)
                    # cv2.imwrite(save_personal_img_left_path, save_img_left)

                    personal_name_cnt += 1

                    # personal_gaze_list.append(gaze)
                    # all_gaze_list.append(gaze)
                    # # print(gaze_cnt)
                    # gaze_cnt += 1


            if all_name_cnt != data_cnt:
                print('all_name_cnt ,all_name_cnt: ',all_name_cnt ,gaze_cnt)
                print(img_path)
                sys.exit()
            

    '''save personal gaze data'''
    save_personal_csv_name = read_folder_name + '.csv'
    save_personal_csv_path = os.path.join(save_csv_folder, save_personal_csv_name)
    print('save personal csv name: ', save_personal_csv_name)
    df = pd.DataFrame(personal_gaze_list)
    df.to_csv(save_personal_csv_path, index=False,header = False)

save_all_csv_name =  'columbia_all.csv'
save_all_csv_path = os.path.join(save_csv_folder, save_all_csv_name)
df = pd.DataFrame(all_gaze_list)
df.to_csv(save_all_csv_path, index=False,header = False)
print('Finish!!')

    # sys.exit()




In [ ]:
import shutil
import re

validation_folder_path = os.path.join(save_img_folder_path, 'columbia_validation')
test_folder_path = os.path.join(save_img_folder_path, 'columbia_test')
os.makedirs(validation_folder_path, exist_ok=True)
os.makedirs(test_folder_path, exist_ok=True)
validation_img_cnt = 1
test_img_cnt = 1

validation_subject = ['0005','0010','0015','0020', '0025', '0030', '0035', '0040', '0045', '0050', '0055']

subject_list = sorted(os.listdir(save_img_folder_path))
print(subject_list)

for subject_name in subject_list:
    print(subject_name)
    if subject_name[0:2] == '00':
        img_folder_path = os.path.join(save_img_folder_path, subject_name, 'left')
        subject_img_list = sorted(os.listdir(img_folder_path), key=lambda s: int(re.search(r'\d+', s).group()))
        print(subject_img_list)

        for subject_img_name in subject_img_list:
            subject_img_path = os.path.join(img_folder_path, subject_img_name)

            if subject_name in validation_subject:
                validation_img_name = str(validation_img_cnt) + '.png'
                validation_img_path = os.path.join(validation_folder_path, validation_img_name)
                shutil.copy(subject_img_path, validation_img_path)
                validation_img_cnt += 1

            else:
                test_img_name = str(test_img_cnt) + '.png'
                test_img_path = os.path.join(test_folder_path, test_img_name)
                shutil.copy(subject_img_path, test_img_path)
                test_img_cnt += 1
print('Finish')


In [ ]:
test_img_path

In [ ]:
import re
print(subject_img_list)
sorted(subject_img_list, key=lambda s: int(re.search(r'\d+', s).group()))


In [ ]:
import re

subject_csv_list = []
validation_data_list = []
test_data_list = []
csv_data_list = []

for csv_name in subject_list:
    if csv_name[0:2] == '00':
        # print(csv_name + '.csv')
        subject_csv_list.append(csv_name + '.csv')

for subject_csv_name in subject_csv_list:
    subject_csv_path = os.path.join(save_csv_folder, subject_csv_name)
    with open(subject_csv_path) as f:
        # validation_data = f.read().split(',')
        csv_datas = re.split(',|\n',f.read())
        csv_datas = csv_datas[:-1]
        # print(np.array(csv_data).shape)
        # csv_data = np.array(csv_data).reshape([4,-1])

        if subject_csv_name[:-4] in validation_subject:
            print(subject_csv_path)
            print(np.array(csv_datas).shape)
            validation_data_list.append(csv_datas)
        else:
            print(subject_csv_path)
            print(np.array(csv_datas).shape)
            # print(csv_data)
            # test_data_list.append(csv_data)
            # csv_data_list.append(csv_data)
            for csv_data in csv_datas:
                test_data_list.append(csv_data)


In [ ]:
print(np.array(csv_data).shape)
print(csv_data[1])

In [ ]:
validation_data = np.array(validation_data_list).reshape([-1,4])
test_data = np.array(test_data_list).reshape([-1,4])



validation_csv_path = os.path.join(save_csv_folder, 'columbia_validation.csv')
test_csv_path = os.path.join(save_csv_folder, 'columbia_test.csv')

df_validation = pd.DataFrame(validation_data)
df_validation.to_csv(validation_csv_path, index=False,header=False)
df_test = pd.DataFrame(test_data)
df_test .to_csv(test_csv_path, index=False, header=False)
print('Finish')            


In [ ]:

print(np.array(validation_data_list).shape)
print(np.array(test_data_list).shape)
# print(np.array(test_gaze_list).shape)

In [ ]:
validation_data_list

In [ ]:
# print(np.array(test_data_list).shape)
print(np.array(test_data_list))
# np.array(test_data_list).reshape([-1,4]).shape

In [ ]:
validation_data.shape